In [29]:
// Data Types and Functions
public record struct Position {
    public long X;
    public long Y;

    public static Position operator+(Position position, Position other) {
        return new Position {
            X = position.X + other.X,
            Y = position.Y + other.Y,
        };
    }
    
    public static Position operator-(Position position, Position other) {
        return new Position {
            X = position.X - other.X,
            Y = position.Y - other.Y,
        };
    }
}

public enum Direction {
    Up,
    Down,
    Left,
    Right
}

public static Position Delta(this Direction direction) {
    return direction switch {
        Direction.Up => new Position { X = 0, Y = -1 },
        Direction.Down => new Position { X = 0, Y = 1 },
        Direction.Left => new Position { X = -1, Y = 0 },
        Direction.Right => new Position { X = 1, Y = 0 },
    };
}

public static readonly var AllDirections = new List<Direction> { Direction.Up, Direction.Down, Direction.Left, Direction.Right };

public record struct Bounds {
    public Position Minimums;
    public Position Maximums;
}

public static Position StepBlizzard(Direction direction, Position position, HashSet<Position> legalPositions, Bounds bounds) {
    var newPosition = position + direction.Delta();
    if (legalPositions.Contains(newPosition) == false) {
        switch (direction) {
            case Direction.Down: newPosition.Y = bounds.Minimums.Y; break;
            case Direction.Up: newPosition.Y = bounds.Maximums.Y; break;
            case Direction.Right: newPosition.X = bounds.Minimums.X; break;
            case Direction.Left: newPosition.X = bounds.Maximums.X; break;
        }
    }
    return newPosition;
}

public class Blizzards {
    public Blizzards(Dictionary<Direction, HashSet<Position>> startingBlizzards) {
        BlizzardsByTimeStep = new Dictionary<long, Dictionary<Direction, HashSet<Position>>> {
            { 0, startingBlizzards }
        };
    }
    private Dictionary<long, Dictionary<Direction, HashSet<Position>>> BlizzardsByTimeStep;

    public Dictionary<Direction, HashSet<Position>> BlizzardsForTimeStep(long timeStep, HashSet<Position> legalPositions, Bounds bounds) {
        if (!BlizzardsByTimeStep.ContainsKey(timeStep)) {
            var blizzards = BlizzardsByTimeStep[timeStep - 1];
            var newBlizzards = blizzards.Select(
                blizzardSet => (
                    Direction: blizzardSet.Key,
                    Positions: blizzardSet.Value
                        .Select(position => StepBlizzard(blizzardSet.Key, position, legalPositions, bounds))
                        .ToHashSet()
                    )
            )
            .ToDictionary(blizzardSet => blizzardSet.Direction, blizzardSet => blizzardSet.Positions);
            BlizzardsByTimeStep[timeStep] = newBlizzards;
        }
        return BlizzardsByTimeStep[timeStep];
    }
}

public record struct State {
    public HashSet<Position> LegalPositions;
    public Position Expedition;
    public Position Goal;
    public long TimeStep;
    public List<Direction?> Steps;
    private Bounds? PrecalclatedBounds;

    public Bounds Bounds() {
        if (!PrecalclatedBounds.HasValue) {
            PrecalclatedBounds = new Bounds {
                Minimums = new Position {
                    X = LegalPositions.Min(p => p.X),
                    Y = LegalPositions.Min(p => p.Y) + 1,
                },
                Maximums = new Position {
                    X = LegalPositions.Max(p => p.X),
                    Y = LegalPositions.Max(p => p.Y) - 1,
                }
            };
        }
        return PrecalclatedBounds.Value;
    }

    public long Priority() {
        var diff = Goal - Expedition;
        return Math.Abs(diff.X) + Math.Abs(diff.Y) + TimeStep;
    }

    public IEnumerable<State> NextStates(Blizzards blizzards) {
        var expeditionCopy = Expedition;
        var possibleNextSteps = AllDirections
            .Select(d => (Step: (Direction?) d, Position: expeditionCopy + d.Delta()))
            .Append((Step: null, Position: expeditionCopy));
        
        var legalPositionsCopy = LegalPositions;
        var boundsCopy = Bounds();
        var newBlizzards =  blizzards.BlizzardsForTimeStep(TimeStep + 1, LegalPositions, Bounds());
        
        var goalCopy = Goal;
        var timeStepCopy = TimeStep;
        var stepsCopy = Steps;
        return possibleNextSteps
            .Where(step => legalPositionsCopy.Contains(step.Position)
                && !newBlizzards.Values.Any(blizzards => blizzards.Contains(step.Position))
            )
            .Select(step => new State {
                LegalPositions = legalPositionsCopy,
                Expedition = step.Position,
                Goal = goalCopy,
                TimeStep = timeStepCopy + 1,
                Steps = stepsCopy.Append(step.Step).ToList()
            });
    }
}

public static (State State, Dictionary<Direction, HashSet<Position>> Blizzards) ParseInput(IEnumerable<string> input) {
    var blizzards = new Dictionary<Direction, HashSet<Position>>();
    foreach (var direction in AllDirections) {
        blizzards.Add(direction, new HashSet<Position>());
    }
    var legalPositions = new HashSet<Position>();
    foreach (var (line, y) in input.Select((line, y) => (line, y))) {
        foreach (var (character, x) in line.Select((c, x) => (c, x))) {
            var position = new Position { X = x, Y = y };
            switch (character) {
                case '#': break;
                case '.': legalPositions.Add(position);
                        break;
                case '^': legalPositions.Add(position);
                        blizzards[Direction.Up].Add(position);
                        break;
                case '>': legalPositions.Add(position);
                        blizzards[Direction.Right].Add(position);
                        break;
                case '<': legalPositions.Add(position);
                        blizzards[Direction.Left].Add(position);
                        break;
                case 'v': legalPositions.Add(position);
                        blizzards[Direction.Down].Add(position);
                        break;
                default: throw new InvalidOperationException($"Unknown character {character} in input");
            }
        }
    }
    return (
        new State {
            LegalPositions = legalPositions,
            Expedition = legalPositions.MinBy(p => p.Y),
            Goal = legalPositions.MaxBy(p => p.Y),
            TimeStep = 0,
            Steps = new List<Direction?>(),
        },
        blizzards
    );
}

public State FindPath(State startingState, Blizzards blizzards) {
    var statesToExplore = new PriorityQueue<State, long>();
    var seenStates = new HashSet<(long, Position)>();
    statesToExplore.Enqueue(startingState, startingState.Priority());
    while (statesToExplore.TryDequeue(out var state, out var _)) {
        if (seenStates.Contains((state.TimeStep, state.Expedition))) {
            continue;
        }
        seenStates.Add((state.TimeStep, state.Expedition));
        if (state.Expedition == state.Goal) {
            return state;
        }
        var candidates = state.NextStates(blizzards)
            .Where(c => !seenStates.Contains((c.TimeStep, c.Expedition)));
        foreach(var candidate in candidates) {
            statesToExplore.Enqueue(candidate, candidate.Priority());
        }
    }
    throw new InvalidOperationException("No path found");
}

In [30]:
// Read input
var input = System.IO.File.ReadAllLines("input.txt");
input

index,value
0,#.########################################################################################################################
1,#.v^<>^^.<^^<v>v>>>.<^<<v.^>vv>>^<<<v>.v<^v^<v<<^<^^<^<^v><^vv.v^^.>^><vv><^<.<^v<>.v<v..^>><v>.vvv.v>^^<^>.<<^<^<^>v.^^<#
2,#.^^<v>><<v><vv<>..<>v<^v><<>><>v.>v.^^<<.v.^<^<vv><v<<v><<^<>^v>v>><>v..v^<^.v><>^>><v<<>v^<<v<v>v.v<v>><<v>^v<v^...<<<>#
3,#><vv>>v>v<^>^^^^vv^<v^<<v<>^<^.^^>^<^vv>^v>><<^^<.>^>v<^v<.v.^<^^v.<.>.<^>v^>.v>>^vv^v.>>>v<<<<v.^^v<^^<^><<<<.^>.>vvv>>#
4,#<.vv>.><<.<^v.>v^v>><v>><<^vv>^v^>.><v>>><^><v<v^^vv>.v^^<v>vv<>v>^v.>>v^>..v<>^v.^>><.vv><^...<>^^>v>vvv>^<^<><v>v<^^^<#
5,#>^.^>vvv>vv>^^vv>v^^v>vv>v^v>^<v>>vvvv^>.v^<v^><^>vvv<>vv<^v<.<^v^>^vv^<><>.^.^>><<^^>^<v^>>>>v><<v^^>^^>>v^v.<^vv^<^v<>#
6,#><^>><vv^vvv.v><v<>^v>^<<<^^<><^<<v<>^<<v><<<v>><v<^^<vv<.>^v>^<v>.>v>>^^>^>><>.<<v<^><>>>^.<vv>.^^<.^^>>v.><<v<>><>v^><#
7,#>>^.<<v.^v>^^v.>v.vv^<<.v<v<..<<v^^>^v.><>v<^^<<.>v>v>v>^v>.<>v<><<^v><^>><^v>v^v>^vvv>>>..>^>>.><v><>.^^v.><<vv<.v>^^<.#
8,#>vv>^^.<>^v<^<<v^>v<^^v><>>>>v^vv>v<^v^><><^.<v^>.<^.v^>><vv>>vvv>^v<<<><vv<v^<<>v>>^v>^^^>^v><^^>^^^<<^^<v<<v<v>^^^<^>>#
9,#>.v^<<.vv<^v>^>>v<>v^v^^v^v<^^<<..<<^v<<<<^<v>^>.<>v<v<v<.<.v>vv^>><^.<.v<^<^v><v<v>.v.><<^>>v^.^vv.vv^.>^^>vv<>^..v>^v<#


In [31]:
// Part 1
var (startingState, startingBlizzards) = ParseInput(input);
var blizzards = new Blizzards(startingBlizzards);
FindPath(startingState, blizzards).TimeStep

269

In [ ]:
// Part 2
var (startingState, startingBlizzards) = ParseInput(input);
var startPosition = startingState.Expedition;
var goal = startingState.Goal;
var blizzards = new Blizzards(startingBlizzards);
var reachGoal = FindPath(startingState, blizzards);
var returnToStart = FindPath(reachGoal with { Goal = startPosition}, blizzards);
var returnToGoal = FindPath(returnToStart with { Goal = goal}, blizzards);
returnToGoal.TimeStep